In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import numpy as np
import string
import csv
import seaborn as sb

from IPython.display import Image


In [1]:
### Preprocessing Data 


def remove_multiple_forecasts(tab_file_name, output_file):
    """
    In: name of tab file, and name of file to return
    Delete delimiter to allow for CSV files
    Takes the latest submission by each contestant and writes to a csv file
    """
    data = pd.read_csv(tab_file_name, parse_dates=['timestamp'], delimiter = '\t')
    q_dat = data.groupby(by=['ifp_id','user_id']).apply(lambda x: x[x.timestamp == x.timestamp.max()])
    q_dat.index = q_dat.index.droplevel().droplevel() #2 droplevels since grouping by 2
    q_dat.to_csv(output_file, index=False)


# tab_file = 'survey_fcasts.yr4.tab'
# output_file = 'survey_fcasts.yr4.pandas.csv'

# remove_multiple_forecasts(tab_file,output_file)

In [6]:
### Helper Functions
def correct_answer(ifp_id):
    # Given an IFP, return its correct answer
    mask = IFP['ifp_id'] == ifp_id
    corr = IFP['outcome'][mask]
    return corr.values.item()

def no_opts(ifp_id):
    # Given an IFP, return the number of options associated
    mask = IFP['ifp_id'] == ifp_id
    no = IFP['n_opts'][mask]
    return no.values.item()

def compute_brier(prob, truth):
    """
    Computes Brier scores according to 
    
    Inputs:
    prob: dims = n_options x 1
    truth: dims = n_options x 1
    
    Outputs: Single value of Brier score for a particular question
    """
    
    if np.shape(prob) != np.shape(truth):
        Warning('Input dimensions must be consistent!')
    
    return np.sum((truth - prob)**2)

def build_outcome_dict(csvfile):
    """This function takes a file (ifps.csv) and builds a dictionary mapping each question
        to the outcome of that question (i.e. {1004-0:'a'})
    """
    outcome_dict = {}
    with open(csvfile, encoding = "ISO-8859-1") as file:  # encoding so python does not throw unicode error
        reader = csv.reader(file)
        for row in reader:
            outcome_dict[row[0]] = row[9]
        outcome_dict.pop("ifp_id", None)
    print("outcome dict built")
    return outcome_dict

def brier_scores(outcome_dict, median_dict):
    """Builds dictionary of brier scores for each question using two dicts as inputs
    """
    brier_dict = {}
    for question in median_dict.keys():
        brier = 0
        for response in median_dict[question].keys():
            if response == outcome_dict[question]:
                brier += (1.0 - median_dict[question][response] ) **2
            elif response != outcome_dict[question]:
                brier += (median_dict[question][response] ) **2
        brier_dict[question] = brier
        #print(question, "added")
    return brier_dict

In [ ]:
### Reading in correct files

forecast_file = 'survey_fcasts.yr2.latest_response.csv'

IFP = pd.read_csv('ifps.csv', encoding='latin1')
ifp_dict = build_outcome_dict('ifps.csv')
data = pd.read_csv(forecast_file)

unique_ifps = data['ifp_id'].unique()


answer_choices = list(string.ascii_lowercase)
df_of_answers = [] #this list will contain dataframes for every answer (e.g. 1001-0 a, 1001-0 b)

for ifp in unique_ifps:
    for i in range(no_opts(ifp)):   #this iterates through every choice that the given ifp has
        letter = answer_choices[i]
        df = data[(data['ifp_id'] == ifp) & (data['answer_option'] == letter)]
        df_of_answers.append(df)


median_outcomes = {}
option_answers = {}
for qa_pair in df_of_answers: # eg 1001a 1001b 1002a 1002b 1002c
    
    ifp_id = qa_pair['ifp_id'].unique()[0]
    ans = qa_pair['answer_option'].unique()[0]
    if ifp_id not in median_outcomes.keys():
        option_answers = {ans : np.median(qa_pair['value'])}
        median_outcomes[ifp_id] = option_answers
    option_answers[ans] = np.median(qa_pair['value'])

median_brier_dict = brier_scores(ifp_dict, median_outcomes)
print('median_brier_dict built with',len(median_brier_dict.values()),'values')
                  
                  #stratify data set with different characteristics and see
                  

In [ ]:

### Population Code dictionary building

def build_population_code_dictionary(data, removeUninformative=False):
    """
    Data is in the form of a dataframe containing the survey_fcasts type file.
    removeUninformative is intended to remove the very low confidence answers
    Must have columns 'ifp_id', 'user_id', 'answer_option', 'value'
    returns dictionary mapping scores to each question. 
    e.g. {'1001-0': {'a': 0.5, 'b': 0.5}, '1076-0': {'a': 0.12, 'b': 0.88}}
    """
    population_outcomes = {}

    for ifp_id, group in data.groupby('ifp_id'): 
        answer_dict = {}     # will be {'a': 144,'b': 232} format
        for user_id, sub_group in group.groupby('user_id'):
            normalised = sub_group['value'] / sub_group['value'].sum()
            c = np.random.choice(list(sub_group['answer_option']),p=list(normalised)) # c = choice
            if c not in answer_dict:
                answer_dict[c] = 1
            else:
                answer_dict[c] += 1
        population_outcomes[ifp_id] = answer_dict
    for question in population_outcomes.keys():
        total  = 0
        for answer in population_outcomes[question].keys():
            total += population_outcomes[question][answer]
        for answer in population_outcomes[question].keys(): # used to be 'all_questions'
            population_outcomes[question][answer] /= total
    return population_outcomes

pop_brier_dict = brier_scores(ifp_dict, population_outcomes)
# pop_brier_dict is a dictionary mapping each question to a brier score
# print('pop_brier_dict built with', len(pop_brier_dict.values()),'values')


# ### Outcome
# 
# Below are the brier scores for both the Population Code and simple Median Aggregation
# 
# 

### Outcome

Below are the brier scores for both the Population Code and simple Median Aggregation



In [ ]:
df = pd.DataFrame(list(pop_brier_dict.items()))
df=df.rename(columns = {0:'ifp_id',1:'Population Code'})
# print(list(pop_brier_dict.values()))
# df['Population Code'] = list(pop_brier_dict.values())
df['Medians'] = list(median_brier_dict.values())

df.head()
print(df['Population Code'].mean())
print(df['Medians'].mean())


In the dataset of first year forecasts, the Population Code aggregation (mean brier = 0.223) is outperformed by the median aggregation (0.195).


Next:
    - Modify scripts to convert json to csv files
    - Swapping forecast from earliest to latest
    - Try population using mode method of aggregation
    - Look at characteristics

In [ ]:
plot = df.boxplot()
plt.show()

Next, Principal Component Analysis on matrix of n questions by m participants. 
First we will attempt analysis on binary questions using the confidence level given for option a.

What will go in the matrix mapping of the question to the participants is the brier score for a given participant- response pair.

In [5]:
import time
forecast_file = 'survey_fcasts.yr1.latest_response.csv'

IFP = pd.read_csv('ifps.csv', encoding='latin1')
#ifp_dict = build_outcome_dict('ifps.csv')
data = pd.read_csv(forecast_file)

unique_ifps = data['ifp_id'].unique()
unique_participants = data['user_id'].unique()


#apply pca both on forecasts and on brier scores
#try to fill values with mean or with baseline .50 scores
#convert to json to use data_loader.py and then you can use the binary? column

### Principal Component Analysis Data Preparation
Here we are trying to create a matrix of user_id and ifp_id. The ifp_id has to be a binary answer. The values in the matrix will be the chance that each person thinks option a will happen.

In [8]:
# data.head()
# df = pd.DataFrame(data['ifp_id'])
# for ifp in unique_ifps:

data_pivot_all_values = pd.pivot_table(data,index=['ifp_id'],columns = ["user_id","answer_option"], values = ["value"])


# binary_data = data_pivot.loc[(data_pivot['value'] == 0.50)]
# df.loc[df['column_name'].isin(some_values)]


# Everything below here works to build the matrix 
data_1 = data.loc[data['answer_option'] == 'a']  


data_pivot = pd.pivot_table(data_1,index=['ifp_id'],columns = ["user_id","answer_option"], values = ["value"])#,aggfunc= lambda x : x)

data_transposed = data_pivot[:100].transpose() #we want the ifp_id to be in the column so we can use the df.mean() function


final_data = data_transposed.fillna(data_transposed.mean())

final_data_transposed = final_data.transpose()
def do_PCA(data):
    from sklearn.decomposition import PCA
    pca = PCA() #initiate a object of the class PCA
    pca.fit_transform(data)
    return pca

pca = do_PCA(final_data_transposed)

# final_data_transposed.head()

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.show()
#this works but still has non binary options in it
#the graph below indicates first component is agreement (explains 70% of variance)

value                                                         \
user_id       3.0       5.0    6.0    15.0   19.0      22.0   23.0   25.0     
answer_option      a         a      a      a      a         a      a      a   
ifp_id                                                                        
1001-0          0.50  0.171670    0.0   0.25   0.60  0.171670   0.20    0.1   
1002-0          0.33  0.141242    0.0   0.30   0.48  0.141242   0.75    0.2   
1003-0          0.50  0.176125    0.0   0.50   0.15  0.100000   0.00    0.2   
1004-0          0.65  0.242804    0.0   0.40   0.10  0.000000   0.00    0.7   
1005-0          0.50  0.740762    1.0   0.70   0.75  0.740762   0.99    0.6   

                                  ...                                   \
user_id          33.0   35.0      ...       5903.0    5907.0    5908.0   
answer_option         a      a    ...            a         a         a   
ifp_id                            ...                                    
1001-0         0.200000   0.50    ...     0.100000  0.050000  0.020000   
1002-0         0.350000   0.50    ...     0.141242  0.141242  0.000000   
1003-0         0.176125   0.05    ...     0.350000  0.000000  0.000000   
1004-0         0.242804   0.10    ...     0.242804  0.242804  0.242804   
1005-0         0.740762   0.60    ...     0.740762  0.740762  0.740762   

                                                                           \
user_id          5915.0    5918.0    5921.0    5924.0    5927.0    5928.0   
answer_option         a         a         a         a         a         a   
ifp_id                                                                      
1001-0         0.171670  0.050000  0.171670  0.150000  0.000000  1.000000   
1002-0         0.141242  0.141242  0.141242  0.000000  0.000000  0.000000   
1003-0         0.176125  0.050000  0.176125  0.120000  0.000000  0.200000   
1004-0         0.242804  0.242804  0.242804  0.242804  0.242804  0.242804   
1005-0         0.740762  0.740762  0.740762  0.740762  0.740762  0.740762   

                         
user_id          5929.0  
answer_option         a  
ifp_id                   
1001-0         0.000000  
1002-0         0.000000  
1003-0         0.000000  
1004-0         0.242804  
1005-0         0.740762  

[5 rows x 1972 columns]

In [24]:
# final_data_transposed.head()
final_data_transposed.shape # ifp ids x features

(100, 1972)

In [11]:
# Below is the variance explained by each of the principal componenets (in cumulative % terms)
print(np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)) 
# big first component indicates large amount of redundant information that can be collapsed into one dimension

[ 68.36  70.77  72.44  73.54  74.43  75.25  76.01  76.73  77.42  78.07
  78.67  79.23  79.76  80.28  80.78  81.27  81.73  82.17  82.6   83.02
  83.43  83.84  84.24  84.63  85.02  85.39  85.76  86.12  86.48  86.83
  87.17  87.51  87.84  88.16  88.47  88.78  89.08  89.37  89.66  89.94
  90.22  90.49  90.76  91.03  91.29  91.55  91.8   92.05  92.3   92.54
  92.77  93.    93.23  93.46  93.68  93.9   94.11  94.32  94.53  94.73
  94.93  95.13  95.32  95.51  95.7   95.88  96.06  96.24  96.41  96.58
  96.74  96.9   97.06  97.22  97.37  97.52  97.67  97.81  97.95  98.09
  98.22  98.35  98.47  98.59  98.71  98.83  98.95  99.06  99.17  99.27
  99.37  99.47  99.56  99.65  99.73  99.8   99.87  99.93  99.98  99.98]


In [ ]:
#data_pivot_filled = data_pivot.fillna(value=new_question_mean_dict,axis=0) #to replace NaN in rows with mean of the row




# pca = do_PCA(data_pivot[:50]) #
# print(pca.explained_variance_ratio_)

# #Try using preformatted json?
# IFP_json = pd.read_json('ifps.json')
# IFP_json.head()

In [ ]:
# NOTE DATA STILL HAS SOME NON BINARY IFPS?

# from data_loader import load_data
# IFP_json = pd.read_json('ifps.json')
# load_data('ifps.json','forecasts.yr1.json','--old')

In [11]:
print(pca.components_[0]) #this first component explains ~ 70% of variance
# pca.components_[0] will be 1972x1 in size for each user_id
#why is mean not 0?

[ 0.00884959  0.02234152  0.03131801 ...,  0.03116568  0.01707274
  0.02709123]


### Principal Component Logistic Regression

Here we are aiming to use the principal components for logistic regression.
We'll train the set of weights using the past data as weights. 
Input to the function will be the set of everybody's prediction for a given question. Output will be overall probability for question.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

IFP = pd.read_csv('ifps.csv', encoding='latin1')

# test_data = final_data_transposed.sample(frac = 0.10)
train, test = train_test_split(final_data_transposed, test_size = 0.1)

def build_target_list(df):
    """input: dataframe with ifp values.
        creates list of [1,0,1,0,0...] corresponding to target values
        """
    targets = []
    for ifp in df.index:
        result = IFP.loc[IFP['ifp_id'] == ifp]
        value = result['outcome'] == 'a'
        if value.item():
            targets.append(1)
        else:
            targets.append(0)
    return targets

total_targets = build_target_list(final_data_transposed)

test_targets = build_target_list(test) 
train_targets = build_target_list(train)

# now need to specify targets using vector of ifp_id results
# are columns and rows around the right way now? yes: features are people and ifp_ids are samples

#To do:
# read documentation on sklearn LogisticRegression
# train logistic regression model on all of data
# use PCA weights to modify dataset (how??)


In [42]:
# EXAMPLE
# >>> from sklearn import datasets, linear_model
# >>> from sklearn.model_selection import cross_val_score
# >>> diabetes = datasets.load_diabetes()
# >>> X = diabetes.data[:150]
# >>> y = diabetes.target[:150]
# >>> lasso = linear_model.Lasso()
# >>> print(cross_val_score(lasso, X, y))  


log_reg = LogisticRegression()
scores = cross_val_score(log_reg, final_data_transposed, total_targets)
print(scores.mean(), scores.std())
# print(cross_val_score(log_reg, final_data_transposed, total_targets))

# log_reg.fit(final_data_transposed, total_targets) #weights = PCA vectors? dimensions wrong??


0.909387997623 0.0657378450129


As is evident, using the cross_val_score function, we get relatively high validation scores on the data alone:

[ 0.97058824  0.93939394  0.81818182]

Next we will try using a dataframe consisting of the first couple PCA components.

In [48]:

contestants = []
for tuple in final_data_transposed.columns.values:
    contestants.append(tuple[1])

pca_df = pd.DataFrame(columns = contestants) # len 1972
for i in range(len(pca.components_)):
     pca_df.loc[i] = pca.components_[i]
pca_df.head()
pca_scores = cross_val_score(log_reg, pca_df, total_targets)
print(pca_scores.mean())
#pca seems to do a much worse job:
# %75.01


0.750148544266


In [120]:
import numpy as np
## Here is attempt at Cross Validation
n = 100 # not sure if this should be no. ifps(100) or no. individuals(1972)
kf_10 = cross_validation.KFold(n,n_folds=10,shuffle=True,random_state=2)
mean_squared_errors = []

score = -1*cross_validationscore = -1*cross_validation.cross_val_score(log_reg, np.ones((n,1)), y.ravel(), cv=kf_10, scoring='mean_squared_error').mean()    
mse.append(score) 

SyntaxError: can't assign to operator (<ipython-input-120-0618da1eecbe>, line 7)

Here is where we see briefly if the predictions by the logistic regression model
are correct.
We take the first row of the train set, and then see simply if the prediction matches
the actual outcome

In [95]:
prediction = log_reg.predict(train.iloc[0])
print('prediction: ',prediction)
result = IFP.loc[IFP['ifp_id'] == '1017-0']
value = result['outcome'] == 'a'
print('actual: ',value.item())

prediction:  [0]
actual:  False


/Users/maxlangenkamp/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [100]:
from sklearn import metrics, cross_validation
predicted = cross_validation.cross_val_predict(LogisticRegression(), train, train_targets, cv=10)
print (metrics.accuracy_score(train_targets, predicted))

0.922222222222
